In [11]:
# %load '../../../global/datasets/WallFollowingRobot.py'
from __future__ import print_function
from PIL import Image
import os
import os.path
import errno
import numpy as np
import sys
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
    
class WallFollowingRobot(data.Dataset):

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00194/sensor_readings_24.data"
    filename = "sensor_readings_24.data"
    md5_checksum = 'ebfec9a1d0a88d231bdba6d33bf1b662'

    def __init__(self, root, transform=None, target_transform=None, download=True):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        self.data = []
        self.labels = []
        fp = os.path.join(root, self.filename)
        data = np.loadtxt(fp, dtype='str', delimiter=',')

        self.data = (data[:, :24]).astype('float32')
        self.labels = data[:, 24]
        self.labels = list(map(lambda x: self.labelToInt(x),self.labels))
        
    def labelToInt(self,label):
    
        if label == 'Move-Forward':
            return 0
        elif label == 'Slight-Right-Turn':
            return 1
        elif label == 'Sharp-Right-Turn':
            return 2
        elif label == 'Slight-Left-Turn':
            return 3
        return None

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        sensor_data, target = self.data[index], self.labels[index]
        
        if self.transform is not None:
            sensor_data = self.transform(sensor_data)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return sensor_data, target

    def __len__(self):
        return len(self.data)

    def _check_integrity(self):
        root = self.root
        fpath = os.path.join(root, self.filename)
        if not check_integrity(fpath, self.md5_checksum):
            return False
        return True

    def download(self):
        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        root = self.root
        download_url(self.url, root, self.filename, self.md5_checksum)


In [12]:
# %load '../../../global/samplers/RandomTrainTestSampler.py'
import torch
import random

class SubsetSampler(object):

    def __init__(self, subset):
        self.subset = subset

    def __iter__(self):
        return iter(self.subset)

    def __len__(self):
        return len(self.subset)


class RandomTrainTestSampler(object):

    def __init__(self, data_source, train_share=0.8):

        # Generate a list of indizes reaching from 0 ... len(data_source)-1
        idxList = list(range(0,len(data_source)))

        # Ensure that list is sorted randomly
        random.shuffle(idxList)

        # Split dataset random shares of train and test data
        numberOfTrainSamples = int(len(data_source) / (1 / train_share))
        
        self.train_samples = idxList[:numberOfTrainSamples]
        self.test_samples = idxList[numberOfTrainSamples:]

    def trainSampler(self):
        return SubsetSampler(self.train_samples)

    def testSampler(self):
        return SubsetSampler(self.test_samples)


In [13]:
import numpy as np
import torch

class IncreaseDimension(object):

    def __call__(self, o):

        return np.expand_dims(o, axis=0)


In [14]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv1d(1, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 11, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 11)
        x = self.classifier(x)
        return x

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets
import torchvision.models as models

transformsData = transforms.Compose([IncreaseDimension()] )

dataset = WallFollowingRobot(root='./data', transform=transformsData, download=True)

sampler = RandomTrainTestSampler(dataset)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=sampler.trainSampler(), num_workers=2 )
testloader = torch.utils.data.DataLoader(dataset, batch_size=20, sampler=sampler.testSampler(), num_workers=2)

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print("Start training")
for epoch in range(300):

    running_loss = 0.0
    for data in trainloader:

        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
  
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


    print(loss.data[0])

    correct = 0
    total = 0
    
    for data in testloader:
  
        inputs, labels = data
        outputs = net(Variable(inputs))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy of the network on the test images: %f %%' % (100 * correct / total))

print('Finished Training')


